<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Практика" data-toc-modified-id="Практика-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Практика</a></span><ul class="toc-item"><li><span><a href="#Загрузка-DataFrame" data-toc-modified-id="Загрузка-DataFrame-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка DataFrame</a></span><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Задача 1</a></span></li></ul></li><li><span><a href="#Теория-вероятности.-События" data-toc-modified-id="Теория-вероятности.-События-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Теория вероятности. События</a></span><ul class="toc-item"><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Задача 3</a></span></li></ul></li></ul></li></ul></div>

# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [213]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [214]:
# Решение
df = df.set_index('Показатель')
df['В норме?'] = np.nan
df_copy = df.copy()
df_copy.Норматив.iloc[1] = 'не более 3'
df_copy.Норматив.iloc[2] = '0'

C:\Users\elena\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [217]:
for i in range(len(df)):
    if ',' in df_copy.Норматив.iloc[i]:
        df_copy.Норматив.iloc[i] = df_copy.Норматив.iloc[i].replace(',','.')

C:\Users\elena\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [218]:
for i in range(len(df)):
    if 'в пределах' in df_copy.Норматив.iloc[i]:
        df_copy.Норматив.iloc[i] = df_copy.Норматив.iloc[i].split()[2].split('-')
        if float(df_copy['Результат анализа'].iloc[i]) > float(df_copy.Норматив.iloc[i][0]):
            if float(df_copy['Результат анализа'].iloc[i]) < float(df_copy.Норматив.iloc[i][1]):
                    df['В норме?'].iloc[i] = 'Да'
        else:
            df['В норме?'].iloc[i] = 'Нет'
    elif 'не более' in df_copy.Норматив.iloc[i]:     
        df_copy.Норматив.iloc[i] = df_copy.Норматив.iloc[i].split()[2]
        if float(df_copy['Результат анализа'].iloc[i]) < float(df_copy.Норматив.iloc[i]):
            df['В норме?'].iloc[i] = 'Да'
        else:
            df['В норме?'].iloc[i] = 'Нет'

In [219]:
df

,Единица измерений,Результат анализа,Норматив,В норме?
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Да
Запах,баллы,1,не более 2-3,Да
Цветность,градусы,б/цвета,не более 30,NaN
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Да
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Да
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Да
Нитраты (по NO3),мг/дм3,24,не более 45,Да
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Да
Хлориды (Cl),мг/дм3,200,не более 350,Да


Вывод: только два показателя выходят за пределы нормы в роднике в Нескучном саду

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [392]:
# Ваше решение
apple = 4
orange = 5
total = 9
count = 0
success = 0
for a in range(1000):
    count = 0
    orange = 5
    for i in range(1, 4):
        if np.random.choice(('orange', 'apple'), 1, orange /(total - i)) == 'orange':
            count += 1
            orange -= 1
    if count == 3:
        success += 1
print(success/1000)

0.125


Экспериментально ответ схож с аналитическим

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [402]:
# Ваше решение
detail = 10
unstadart = 3
success = 0
for a in range (1000):
    count = 0
    standart = detail - unstadart
    for i in range(1, 3):
        if np.random.choice(('standart', 'unstandart'), 1, standart / (detail - i)) == 'standart':
            count += 1
            standart -= 1
    if count == 2:
        success += 1
print(success/1000)

0.228
